# Location stopword list creating using the ACLED dataset

We extract location words from the 'country', 'admin1', 'admin2', and 'admin3' columns of the ACLED dataset and combine them together to make a stopwords list. For n-grams such that n > 1, e.g. 'addis abeba', both bigram 'addis abeba' and unigram 'addis' and 'abeba' are included in this list.

In [1]:
import numpy as np
import pandas as pd

In [2]:
acled_df = pd.read_csv(
        "/home/aadelucia/files/minerva/data/2014-01-01-2020-01-01_acled_reduced_all.csv",
        keep_default_na=False,  # Preserve "NA" country code
        parse_dates=[4],  # Event dates
    ).set_index(["iso3", "event_date"])
acled_df

data_id  iso event_id_cnty  event_id_no_cnty  year  \
iso3 event_date                                                       
SDN  2020-01-01  6713858  729      SUD13572             13572  2020   
     2020-01-01  6713859  729      SUD13573             13573  2020   
     2020-01-01  6713863  729      SUD13566             13566  2020   
     2020-01-01  6713868  729      SUD13567             13567  2020   
     2020-01-01  6713883  729      SUD13570             13570  2020   
...                  ...  ...           ...               ...   ...   
PAK  2014-01-01  5937857  586      PAK21260             21260  2014   
     2014-01-01  5940950  586      PAK21255             21255  2014   
     2014-01-01  5937645  586      PAK21254             21254  2014   
CAF  2014-01-01  6706670  140       CEN1636              1636  2014   
EGY  2014-01-01  5262838  818       EGY4422              4422  2014   

                 time_precision event_type             sub_event_type  \
iso3 event_date                                                         
SDN  2020-01-01               1   Protests           Peaceful protest   
     2020-01-01               1   Protests           Peaceful protest   
     2020-01-01               1   Protests  Protest with intervention   
     2020-01-01               1   Protests           Peaceful protest   
     2020-01-01               1   Protests           Peaceful protest   
...                         ...        ...                        ...   
PAK  2014-01-01               1   Protests           Peaceful protest   
     2014-01-01               2   Protests           Peaceful protest   
     2014-01-01               1   Protests           Peaceful protest   
CAF  2014-01-01               1      Riots      Violent demonstration   
EGY  2014-01-01               1   Protests  Protest with intervention   

                                             actor1  \
iso3 event_date                                       
SDN  2020-01-01                  Protesters (Sudan)   
     2020-01-01                  Protesters (Sudan)   
     2020-01-01                  Protesters (Sudan)   
     2020-01-01                  Protesters (Sudan)   
     2020-01-01                  Protesters (Sudan)   
...                                             ...   
PAK  2014-01-01               Protesters (Pakistan)   
     2014-01-01               Protesters (Pakistan)   
     2014-01-01               Protesters (Pakistan)   
CAF  2014-01-01  Rioters (Central African Republic)   
EGY  2014-01-01                  Protesters (Egypt)   

                                                     assoc_actor_1  ...  \
iso3 event_date                                                     ...   
SDN  2020-01-01                                                     ...   
     2020-01-01                              Refugees/IDPs (Sudan)  ...   
     2020-01-01  Masalit Ethnic Group (Sudan); Refugees/IDPs (S...  ...   
     2020-01-01                              Refugees/IDPs (Sudan)  ...   
     2020-01-01                                                     ...   
...                                                            ...  ...   
PAK  2014-01-01                                                     ...   
     2014-01-01                                                     ...   
     2014-01-01                                                     ...   
CAF  2014-01-01            Muslim Group (Central African Republic)  ...   
EGY  2014-01-01                                   Students (Egypt)  ...   

                       admin3           location latitude  longitude  \
iso3 event_date                                                        
SDN  2020-01-01                       Wad Medani  14.4004    33.5184   
     2020-01-01                           Zamzam  13.4840    25.3075   
     2020-01-01                Ardamata IDP Camp  13.4757    22.4931   
     2020-01-01                          Gereida  11.2754    25.1403   
     2020-01-01    

In [3]:
acled_df.iloc[0]

data_id                                                       6713858
iso                                                               729
event_id_cnty                                                SUD13572
event_id_no_cnty                                                13572
year                                                             2020
time_precision                                                      1
event_type                                                   Protests
sub_event_type                                       Peaceful protest
actor1                                             Protesters (Sudan)
assoc_actor_1                                                        
inter1                                                              6
actor2                                                               
assoc_actor_2                                                        
inter2                                                              0
interaction         

In [4]:
acled_df.iloc[1]

data_id                                                       6713859
iso                                                               729
event_id_cnty                                                SUD13573
event_id_no_cnty                                                13573
year                                                             2020
time_precision                                                      1
event_type                                                   Protests
sub_event_type                                       Peaceful protest
actor1                                             Protesters (Sudan)
assoc_actor_1                                   Refugees/IDPs (Sudan)
inter1                                                              6
actor2                                                               
assoc_actor_2                                                        
inter2                                                              0
interaction         

In [5]:
acled_df.iloc[1000]

data_id                                                       6688043
iso                                                               764
event_id_cnty                                                 THA7466
event_id_no_cnty                                                 7466
year                                                             2019
time_precision                                                      1
event_type                                                   Protests
sub_event_type                                       Peaceful protest
actor1                                          Protesters (Thailand)
assoc_actor_1                                                        
inter1                                                              6
actor2                                                               
assoc_actor_2                                                        
inter2                                                              0
interaction         

In [19]:
countries = acled_df['country'].apply(lambda x: x.lower()).unique()
locs = acled_df['location'].apply(lambda x: x.lower()).unique()

In [26]:
ad3 = acled_df['admin3'].apply(lambda x: x.lower()).unique()
ad3

array(['', 'mardan', 'sujawal', ..., 'tezo', 'bhairab', 'gaturi'],
      dtype=object)

In [20]:
concated = np.concatenate((countries, locs))

In [33]:
def get_set(acled_df, col_list):
    arr = acled_df[col_list[0]].apply(lambda x: x.lower()).unique()
    col_list = col_list[1:]
    for col in col_list:
        arr = np.concatenate((arr, acled_df[col].apply(lambda x: x.lower()).unique()))
    s = set(arr)
    if '' in s:
        s.remove('') # remove empty string
    return s

In [34]:
col_list = ['country', 'admin1', 'admin2', 'admin3']
agg_set = get_set(acled_df, col_list)

In [35]:
len(agg_set)

7494

In [37]:
list(agg_set)[:100]

['luanshya',
 'mithapukur',
 'khemis el khechna',
 'bafang',
 'bangourain',
 'blue crane route',
 'medea',
 'sri lanka',
 'ogbomosho north',
 'gampaha',
 'buhaguzi',
 'arua hill',
 'bizerte',
 'moramanga',
 'mossel bay',
 'magbema',
 'preaek prasab',
 'mbouda',
 'ibiono ibom',
 'kokona',
 'anezi',
 'kyankwanzi',
 'el-ibrahimiya',
 'upper denkyira west',
 'nakhon sawan',
 'chorfa',
 'sakhisizwe',
 'takaba',
 'simbandi brassou',
 'rakai',
 'ilebo',
 'su-ngai kolok',
 'ambohidratrimo',
 'shagamu',
 'tarash',
 'aweil north',
 'dadeldhura',
 'maroantsetra',
 'matli',
 'oyo east',
 'kadiogo',
 'ruhaama',
 'tichy',
 'larache',
 'bamnet narong',
 'butanuka',
 'mustang',
 'ruwenzori',
 'uriri',
 'tiebele',
 'erute',
 'dera bugti',
 'butaganzwa',
 'laikipia west',
 'nongoa',
 'kipangani',
 'sumbe',
 'kinieran',
 'sebdou',
 'tazemmourt',
 'quaidabad',
 'tapac',
 'dar el beida',
 'talat kwan',
 'kwango',
 'ba-phalaborwa',
 'mpanda',
 'markz abu qurqas',
 'bakool',
 'hamar jabjab',
 'khairpur tamew

In [46]:
agg_set_backup = agg_set.copy()

In [56]:
agg_set = agg_set_backup.copy()
split_list = []
combine_list = []
for entry in list(agg_set):
    tokens = entry.split(" ")
    for token in tokens:
        split_list.append(token)
    combine_list.append("".join(tokens))
agg_set.update(split_list)
agg_set.update(combine_list)
len(agg_set)

11781

In [58]:
'addisababa' in agg_set

True

# Resulting file is saved to /home/jzhan237/files/data/location_words.txt

In [59]:
with open('/home/jzhan237/files/data/location_words.txt', 'w') as f:
    for entry in agg_set:
        f.write(f'{entry}\n')